#  Multistage Compression Refrigeration Cycle

* SimVCCE Branch :B2023-2

## 1 The Two-stage Compression Refrigeration Cycle

>Yunus A. Cengel, Michael A. Boles, Thermodynamics: An Engineering Approach, 8th Edition,McGraw-Hill, 2015
>
>EXAMPLE 11–4: Multistage Compression Refrigeration: Page627-628

**EXAMPLE 11–5 Two-Stage Refrigeration Cycle with a Flash Chamber,Page627-628**

Consider a two-stage compression refrigeration system operating between the pressure limits of 0.8 and 0.14 MPa. 

![two stage](./img/vcr/two_stage.jpg)

The working fluid is  R134a.

The refrigerant leaves the condenser as a saturated liquid and is throttled to a flash chamber operating at 0.32 MPa. 

Part of the refrigerant evaporates during this flashing process, and this vapor is mixed with the refrigerant leaving the low-pressure compressor. 

The mixture is then compressed to the condenser pressure by the high-pressure compressor. 

The liquid in the flash chamber is throttled to the evaporator pressure and cools the refrigerated space as it vaporizes in the evaporator. 

Assuming the refrigerant leaves the evaporator as a saturated vapor and both compressors are isentropic,

**Determine**

* (a) the fraction of the refrigerant that evaporates as it is throttled to the flash chamber,
* (b) the amount of heat removed from the refrigerated space and the compressor work per unit mass of refrigerant flowing through the condenser, and
* (c) the coefficient of performance.


## 2 The Analysis


### 2.1 The New Device Classes

* Flash Chamber

* Mixing Chamber

#### 2.1.1  Flash Chamber

In [ ]:
# %load ../../SimVCCE/vccpython/components/flashchamber.py

"""
class FlashChamber
                         ↓   iPort 
                   ┌─────┴─────┐
                   │           │
                   │           │→  oPortV
                   │────────── │
                   └─────┬─────┘
                         ↓  oPortL 
json example:
      {
            "name": "flashchamber",
            "devtype": "FLASH_CHAMBER",
            "iPort": {
                "p": 0.6
            },
            "oPortL": {
                "x": 0.0
            },
            "oPortV": {  "x": 1.0
            }
        },
     
"""
from components.port import Port

class FlashChamber:
    energy = "none"
    devtype = "FLASH_CHAMBER"

    def __init__(self, dictDev):
        """
        Initializes lashChamber
        """
        self.name = dictDev['name']
        self.iPort = Port(dictDev['iPort'])
        self.oPortV = Port(dictDev['oPortV'])
        self.oPortL = Port(dictDev['oPortL'])

    def state(self):
        if self.iPort.p is not None:
            self.oPortV.p = self.iPort.p
            self.oPortL.p = self.oPortL.p
            
        elif self.oPortL.p is not None:
            self.oPortV.p = self.oPortV.p
            self.iPort.p = self.iPort.p
       
        elif self.oPortV.p is not None:
            self.oPortL.p = self.oPortL.p
            self.iPort.p = self.iPort.p
      
    def balance(self):
        """flash chamber """
        oPortV_fdot =self.iPort.x
        oPortL_fdot =1.0-self.iPort.x
        self.oPortV.mdot = self.iPort.mdot*oPortV_fdot
        self.oPortL.mdot = self.iPort.mdot*oPortL_fdot
    
       
    def __str__(self):
        result = '\n' + self.name
        result += '\n' + "  PORT  " + Port.title
        result += '\n' + "  iPort "+self.iPort.__str__()
        result += '\n' + " oPortV " + self.oPortV.__str__()
        result += '\n' + " oPortL " + self.oPortL.__str__()
        return result


#### 2.1.2 Mixing Chamber

In [ ]:
# %load ../../SimVCCE/vccpython/components/mixingchamber.py
"""
MixingChamber
                         ↑ oPort 
                   ┌─────┴─────┐
                   │           │
                 → │           │ 
          iPort0   │           │
                   └─────┬─────┘
                         ↑ iPort1 
 
 json example:
  {
            "name": "mixingchamber",
            "devtype": "MIXING_CHAMBER",
            "iPort0": {
                "x": 1.0
            },
            "iPort1": {
            },
            "oPort": {
                "p": 0.6
            }
        }
"""

from components.port import Port


class MixingChamber:
    energy = "none"
    devtype = "MIXING_CHAMBER"

    def __init__(self, dictDev):
        """
        Initializes Merge_Two2one
        """
        self.name = dictDev['name']
        self.iPort0 = Port(dictDev['iPort0'])
        self.iPort1 = Port(dictDev['iPort1'])
        self.oPort = Port(dictDev['oPort'])

    def state(self):
        if self.iPort0.p is not None:
            self.oPort.p = self.iPort0.p
            self.iPort1.p = self.iPort0.p
        elif self.iPort1.p is not None:
            self.iPort0.p = self.iPort1.p
            self.oPort.p = self.iPort1.p
        elif self.oPort.p is not None:
            self.iPort0.p = self.oPort1.p
            self.iPort1.p = self.oPort1.p

    def balance(self):
        self.oPort.mdot = self.iPort0.mdot+self.iPort1.mdot
        self.oPort.h = (self.iPort0.mdot*self.iPort0.h +
                           self.iPort1.mdot*self.iPort1.h)/self.oPort.mdot

    def __str__(self):
        result = '\n' + self.name
        result += '\n' + "  PORT  " + Port.title
        result += '\n' + " iPort0 "+self.iPort0.__str__()
        result += '\n' + " iPort1 " + self.iPort1.__str__()
        result += '\n' + "  oPort " + self.oPort.__str__()
        return result


#### 2.1.3 `__init__.py`



In [ ]:
# %load ../../SimVCCE/vccpython/components/__init__.py
"""
General Object-oriented Abstraction of VC Cycle 

   Components Package  : port and devices
   
 Author: Cheng Maohua cmh@seu.edu.cn    
"""

from .compressor import Compressor
from .condenser import Condenser
from .expansionvalve import ExpansionValve
from .evaporator import Evaporator
# add the new device class
from .flashchamber import FlashChamber
from .mixingchamber import MixingChamber


# ------------------------------------------------------------------------------
# compdict
#  typedev: class
#     Note: add  typedev: class to the dict after you add the new device class
# --------------------------------------------------------------------------------

compdict = {
    Compressor.devtype: Compressor,
    Condenser.devtype: Condenser,
    ExpansionValve.devtype: ExpansionValve,
    Evaporator.devtype: Evaporator,
    #  add the new device class
    FlashChamber.devtype:  FlashChamber,
    MixingChamber.devtype:  MixingChamber
}



### 2.2 The Modified  Sequential-modular Approach

 * Branch 2023-2  
 
**Sequential-modular approach(SM 序贯模块法):**

* Process **units** are solved in  <b style="color:blue">sequence</b>

```python
def __component_simulator(self):
        # 2 the ports state of device
        for key in self.comps:
            self.comps[key].state()

        # 3 the nodes state of connectors
        for item in self.conns.nodes:
            if item.stateok == False:
                item.state()
        # 4 comps[key].balance()
        for curdev in self.comps:
            self.comps[curdev].balance()

```
---
But,there are **dependencies** of the **mass** float rate and **port's state** calculation.

For example: The **Mixing Chamber** in a two-stage compression refrigeration cycle

* The **fraction** of the refrigerant is not known from **Flash Chamber**

* The **inlet refrigerant state** from **low-pressure compressor** is not known

then,

* the outlet refrigerant state of the **mixing chamber**  is not known

* the outlet  refrigerant state  **high-pressure  compressor** is not known

![two stage](./img/vcr/two_stage.jpg)

If we do **not** have the **mass flow rate** or **port's states** of the device, t

**The simple sequence may be failed!**



In the example, we provide one general method to deal with  the mass flow rate and port's state fails

```python
  def __component_simulator(self):
        state_nodes = self.conns.nodes.copy()

        keys = list(self.comps.keys())
        deviceok = False
        CountsDev = len(self.comps)
        i = 0  # i: the count of deviceok to avoid endless loop
        while (deviceok == False and i <= CountsDev):
            for curdev in keys:
                try:
                    # step 2: the port state: thermal process
                    self.comps[curdev].state()

                    # step 3  the port state: new port's parameter pairs
                    for port in state_nodes:
                        if port.stateok == False:
                            port.state()
                            if port.state() == True:
                                state_nodes.remove(port)

                    # step 4: the port state ：the energy and mass balance
                    self.comps[curdev].balance()
                    keys.remove(curdev)
                except:
                    pass

            i += 1
            if (len(keys) == 0):
                deviceok = True

        if len(keys) > 0:
            print(keys)  # for debug

```

In [ ]:
# %load ../../SimVCCE/vccpython/vcc/vccobj.py
"""
General Object-oriented Abstraction of VC Cycle 

 class VCCycle: the Simulator class of VC Cycle  

 Author: Cheng Maohua cmh@seu.edu.cn
"""

from time import time, localtime, strftime
from getpass import getuser

from components import compdict
from components.port import Port
from .connector import Connector


class VCCycle:

    def __init__(self, dictcycle):
        """
          dictcycle={"name":namestring,
                     "refrigerant":refrigerantstring,
                     "components":[{component1},{component2},...],
                     "connectors":{"name1.port1":"name2.port2",...}
                  }
          TO:     
             self.comps : dict of all component objects      
             self.conns : the connector object
        """
        self.name = dictcycle["name"]
        self.cycle_refrigerant = dictcycle["refrigerant"]
        Port.cycle_refrigerant = self.cycle_refrigerant

        # 1 convert dict to the dict of device objects: {device name:device obiect}
        self.comps = {}
        for curdev in dictcycle["components"]:
            self.comps[curdev['name']] = compdict[curdev['devtype']](curdev)

        # 2 set the nodes value and alias between the item of nodes and the port of devices
        self.conns = Connector(dictcycle["connectors"], self.comps)
    
    def __component_simulator(self):
        state_nodes = self.conns.nodes.copy()

        keys = list(self.comps.keys())
        deviceok = False
        CountsDev = len(self.comps)
        i = 0  # i: the count of deviceok to avoid endless loop
        while (deviceok == False and i <= CountsDev):
            for curdev in keys:
                try:
                    # step 2: the port state: thermal process
                    self.comps[curdev].state()

                    # step 3  the port state: new port's parameter pairs
                    for port in state_nodes:
                        if port.stateok == False:
                            port.state()
                            if port.state() == True:
                                state_nodes.remove(port)

                    # step 4: the port state ：the energy and mass balance
                    self.comps[curdev].balance()
                    keys.remove(curdev)
                except:
                    pass

            i += 1
            if (len(keys) == 0):
                deviceok = True

        if len(keys) > 0:
            print(keys)  # for debug

    def simulator(self):
        self.__component_simulator()

        self.Wc = 0.0
        self.Qin = 0.0
        self.Qout = 0.0

        for key in self.comps:
            if self.comps[key].energy == "CompressionWork":
                self.Wc += self.comps[key].Wc
            elif self.comps[key].energy == "QIN":
                self.Qin += self.comps[key].Qin
            elif self.comps[key].energy == "QOUT":
                self.Qout += self.comps[key].Qout

        self.cop = self.Qin / self.Wc
        self.cop_hp = self.Qout / self.Wc

    def __str__(self):
        curtime = strftime("%Y/%m/%d %H:%M:%S", localtime(time()))
        result = f"\nThe Vapor-Compression Cycle: {self.name} ({curtime} by {getuser()})\n"
        result += f"\nRefrigerant: {self.cycle_refrigerant}\n"
        
        rusult_items = {'Compression Work(kW): ': self.Wc,
                        'Refrigeration Capacity(kW): ': self.Qin,
                        '\tCapacity(ton): ': self.Qin*60*(1/211),
                        'The heat transfer rate(kW): ': self.Qout,
                        'The coefficient of performance: ': self.cop,
                        'The coefficient of performance(heat pump):': self.cop_hp}
        for name, value in rusult_items.items():
            result += f'{name:>35} {value:{">5.2f" if type(value) is float else ""}}\n'
        return result



### 2.3 The JOSN of Cycles


In [ ]:
# %load ../../SimVCCE/vccpython/jsonmodel/vcr_two_stage_11_5.json
{
    "name": "The_TWO_STAGE_11_5",
    "refrigerant": "R134a",
    "components": [
        {
            "name": "Compressor_HP",
            "devtype": "COMPRESSOR",
            "iPort": {
                "p": 0.32,
                "mdot": 1.0
            },
            "oPort": {
                "p": 0.8
            },
            "ef": 1.0
        },
        {
            "name": "Compressor_LP",
            "devtype": "COMPRESSOR",
            "iPort": {
                "p": 0.14
            },
            "oPort": {
                "p": 0.32
            },
            "ef": 1.0
        },
        {
            "name": "ExpansionValve1",
            "devtype": "EXPANSIONVALVE",
            "iPort": {
                "p": 0.8,
                "x": 0.0
            },
            "oPort": {
                "p": 0.32
            }
        },
        {
            "name": "ExpansionValve2",
            "devtype": "EXPANSIONVALVE",
            "iPort": {
                "p": 0.32
            },
            "oPort": {}
        },
        {
            "name": "Condenser",
            "devtype": "CONDENSER",
            "iPort": {
                "p": 0.8
            },
            "oPort": {
                "p": 0.8,
                "x": 0.0
            }
        },
        {
            "name": "Flash_Chamber",
            "devtype": "FLASH_CHAMBER",
            "iPort": {
                "p": 0.32
            },
            "oPortL": {
                "p": 0.32,
                "x": 0.0
            },
            "oPortV": {
                "p": 0.32,
                "x": 1.0
            }
        },
        {
            "name": "Evaporator",
            "devtype": "EVAPORATOR",
            "iPort": {},
            "oPort": {
                "p": 0.14,
                "x": 1.0
            }
        },
        {
            "name": "Mixing_Chamber",
            "devtype": "MIXING_CHAMBER",
            "iPort0": {
                "p": 0.32,
                "x": 1
            },
            "iPort1": {
                "p": 0.32
            },
            "oPort": {
                "p": 0.32
            }
        }
    ],
    "connectors": {
        "Compressor_HP.oPort": "Condenser.iPort",
        "Condenser.oPort": "ExpansionValve1.iPort",
        "ExpansionValve1.oPort": "Flash_Chamber.iPort",
        "Flash_Chamber.oPortV": "Mixing_Chamber.iPort0",
        "Flash_Chamber.oPortL": "ExpansionValve2.iPort",
        "ExpansionValve2.oPort": "Evaporator.iPort",
        "Evaporator.oPort": "Compressor_LP.iPort",
        "Compressor_LP.oPort": "Mixing_Chamber.iPort1",
        "Mixing_Chamber.oPort": "Compressor_HP.iPort"
    }
}

### 2.4 The Main Application

* vccapp_json.py  


In [ ]:
# %load ../../SimVCCE/vccpython/vccapp_json.py

"""
General Object-oriented Abstraction  of VC Cycle 

Yunus A. Cengel, Michael A. Boles, Thermodynamics: An Engineering Approach, 8th Edition,McGraw-Hill, 2015.


The Simulator of VC Cycle 
  * Input :the json file of the cycle model
  * Output: text file
Run: 
   python vccapp_json.py

 Author: Cheng Maohua cmh@seu.edu.cn

"""
import json
import os
import glob
from vcc.vccobj import VCCycle
from vcc.utils import OutFiles

if __name__ == "__main__":
    curpath = os.path.abspath(os.path.dirname(__file__))
    ResultFilePath = curpath+'/result/'

    json_filenames_str = curpath+'\\'+'./jsonmodel/*.json'
    json_filenames = glob.glob(json_filenames_str)

    for i in range(len(json_filenames)):
        with open(json_filenames[i], 'r') as f:
            thedictcycle = json.loads(f.read())

        # the simulator
        cycle = VCCycle(thedictcycle)
        cycle.simulator()
        # output to console
        OutFiles(cycle)
        # output to the file
        ResultFileName = ResultFilePath+thedictcycle['name']
        OutFiles(cycle, ResultFileName + '.txt')


In [ ]:
!python ../../SimVCCE/vccpython/vccapp_json.py 

## 3 glob-  Unix style pathname pattern expansion

* https://docs.python.org/3/library/glob.html

The `glob` module finds all the **pathnames** matching a specified **pattern** according to the rules used by the Unix shell,

**pattern**: *

In [ ]:
import glob
glob.glob('./Unit4*.ipynb')

patter:[2-4]

In [ ]:
glob.glob('./Unit4-[2-4]-*.ipynb')

**patter:** ?

In [ ]:
glob.glob('./Unit4-?-RefrigerationCycle*.ipynb')